In [1]:
import matplotlib.pyplot as plt
from math import isclose
from sklearn.decomposition import PCA
import os
import networkx as nx
import numpy as np
import pandas as pd
from stellargraph import StellarGraph, datasets
from stellargraph.data import EdgeSplitter
from collections import Counter
import multiprocessing
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from py2neo import Graph


%matplotlib inline

In [2]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo4jneo4j"))

## StellarGraph

Stellargraph seen not to work as the ID and the label are seen not to be unique.

In [3]:
query = """
MATCH (p:Person)
RETURN p.label as label, p.trianglesTrain1 as triangles
"""

nodes = graph.run(query).to_data_frame()

In [4]:
query = """
MATCH (p:Person)-[r:INTERACTS_1]->(p2:Person)
RETURN p.id as source, p2.id as target, r.weight as weight
"""

weighted_edges = graph.run(query).to_data_frame()

In [5]:
nodes.set_index('label', inplace=True)

In [6]:
nodes

,triangles
label,
Robett,0
Smalljon,0
Edmures Son,0
Rooses Son,0
Talla,0
...,...
Unsullied Captain,0
Vale Lord,0
Vicky,0


In [7]:
weighted_edges

,source,target,weight
0,ADDAM_MARBRAND,TYWIN,2
1,ADDAM_MARBRAND,LEO_LEFFORD,3
2,ADDAM_MARBRAND,KEVAN,3
3,AEGON,MAESTER_AEMON,2
4,AEGON,AERYS,2
...,...,...,...
544,TYRION,TYSHA,2
545,TYRION,VARYS,2
546,VARDIS_EGEN,TYRION,9
547,VARYS,YOREN,3


In [8]:
weighted_homogeneous = StellarGraph(nodes, weighted_edges)
print(weighted_homogeneous.info())

ValueError: expected IDs to appear once, found some that appeared more: 'Lommy', 'Qhorin', 'Wolkan'

## Graphsage embedding 

In [12]:
# We need to drop the graph if it already exists

query1 = """
CALL gds.graph.drop('myGraph1') YIELD graphName;
"""

query2 = """
CALL gds.graph.drop('myGraph2') YIELD graphName;
"""

query3 = """
CALL gds.graph.drop('myGraph3') YIELD graphName;
"""

query4 = """
CALL gds.graph.drop('myGraph4') YIELD graphName;
"""

query5 = """
CALL gds.graph.drop('myGraph5') YIELD graphName;
"""

query6 = """
CALL gds.graph.drop('myGraph6') YIELD graphName;
"""

query7 = """
CALL gds.graph.drop('myGraph7') YIELD graphName;
"""

query8 = """
CALL gds.graph.drop('myGraph8') YIELD graphName;
"""

graph.run(query1)
graph.run(query2)
graph.run(query3)
graph.run(query4)
graph.run(query5)
graph.run(query6)
graph.run(query7)
graph.run(query8)

graphName
myGraph8


In [13]:
# Make the in memory graphs for adding triangle counts and clustering coefficients
query1 = """
CALL gds.graph.project(
  'myGraph1',
  'Person',
  {
    INTERACTS_1: {
      properties: 'weight',
      orientation: 'UNDIRECTED'
    }
  }
)
"""

query2 = """
CALL gds.graph.project(
  'myGraph2',
  'Person',
  {
    INTERACTS_2: {
    properties: 'weight',
    orientation: 'UNDIRECTED'
}
}
)
"""
query3 = """
CALL gds.graph.project(
  'myGraph3',
  'Person',
  {
    INTERACTS_3: {
    properties: 'weight',
    orientation: 'UNDIRECTED'
}
  }
)
"""
query4 = """
CALL gds.graph.project(
  'myGraph4',
  'Person',
  {
    INTERACTS_4: {
    properties: 'weight',
    orientation: 'UNDIRECTED'
}
  }
)
"""
query5 = """
CALL gds.graph.project(
  'myGraph5',
  'Person',
  {
    INTERACTS_5: {
    properties: 'weight',
    orientation: 'UNDIRECTED'
}
  }
)
"""
graph.run(query1)
graph.run(query2)
graph.run(query3)
graph.run(query4)
graph.run(query5)


nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{Person: {label: 'Person', properties: {}}}","{INTERACTS_5: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACTS_5', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}}",myGraph5,418,866,50


In [14]:
# Make the in memory graphs for adding triangle counts and clustering coefficients
query6 = """
CALL gds.graph.project(
  'myGraph6',
  'Person',
  {
    INTERACTS_6: {
      properties: 'weight',
      orientation: 'UNDIRECTED'
    }
  }
)
"""

query7 = """
CALL gds.graph.project(
  'myGraph7',
  'Person',
  {
    INTERACTS_7: {
      properties: 'weight',
      orientation: 'UNDIRECTED'
    }
  }
)
"""

query8 = """
CALL gds.graph.project(
  'myGraph8',
  'Person',
  {
    INTERACTS_8: {
      properties: 'weight',
      orientation: 'UNDIRECTED'
    }
  }
)
"""

graph.run(query6)
graph.run(query7)
graph.run(query8)


nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{Person: {label: 'Person', properties: {}}}","{INTERACTS_8: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACTS_8', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}}",myGraph8,418,1200,34


In [15]:
query = """ 
CALL gds.degree.mutate(
  'myGraph1',
  {
    mutateProperty: 'degree'
  }
) 
YIELD nodePropertiesWritten
"""

graph.run(query)

nodePropertiesWritten
418


In [16]:
query = """
CALL gds.triangleCount.mutate(
    'myGraph1',
    {
        mutateProperty: 'trianglecount'
    }
)
YIELD nodePropertiesWritten
"""

graph.run(query)

nodePropertiesWritten
418


In [17]:
query = """
CALL gds.localClusteringCoefficient.mutate(
    'myGraph1',
    {
        mutateProperty: 'clusteringcoefficient'
    }
)
YIELD nodePropertiesWritten
"""

graph.run(query)

nodePropertiesWritten
418


In [24]:
query = """ 
CALL gds.beta.graphSage.train(
  'myGraph1',
  {
    modelName: 'myModel3',
    featureProperties: ['degree','trianglecount', 'clusteringcoefficient'],
    relationshipTypes: ['INTERACTS_1'],
    relationshipWeightProperty: 'weight',
    embeddingDimension: 120,
    randomSeed: 19
  }
)
YIELD trainMillis
RETURN trainMillis
"""

graph.run(query)

trainMillis
671


In [27]:
query = """ 
CALL gds.beta.graphSage.stream(
  'myGraph1',
  {
    modelName: 'myModel3'
  }
)
YIELD nodeId, embedding
"""

a = graph.run(query).to_data_frame()

In [28]:
a['embedding'][1]

[0.13483924895325158,
 2.71071429790426e-12,
 1.845072383668191e-12,
 1.5579567907303607e-12,
 0.13483496305229134,
 2.06890470627865e-12,
 1.5090699388769543e-12,
 0.13484180559403516,
 1.5494056108974582e-12,
 2.0636960461856468e-12,
 1.0617861145221734e-07,
 0.1348418055921667,
 0.13484176042634835,
 1.5410626172102875e-12,
 0.1348381921622749,
 0.13484130611835682,
 0.1348417755390128,
 0.13483397331789718,
 0.1348417787843348,
 0.13483885754668484,
 3.0208984932527384e-12,
 2.2841452417601484e-12,
 1.7504903628648787e-12,
 0.13484145258426428,
 0.13484178367525435,
 0.13484180559477832,
 0.13483438498162822,
 2.336073272928913e-12,
 2.5395882397620416e-12,
 0.13484177809313813,
 0.1348351615406408,
 0.13484109003998637,
 0.13483244935222688,
 2.039146628427716e-12,
 0.13484178943281605,
 0.13483859850153793,
 0.1348410163959127,
 2.236735163552774e-12,
 0.13484180550104902,
 1.4446270476261423e-12,
 0.1348368391879147,
 1.6974377333154793e-12,
 0.13484167929029306,
 1.660201426573

In [29]:
a['embedding'][2]


[0.13483924895325158,
 2.71071429790426e-12,
 1.845072383668191e-12,
 1.5579567907303607e-12,
 0.13483496305229134,
 2.06890470627865e-12,
 1.5090699388769543e-12,
 0.13484180559403516,
 1.5494056108974582e-12,
 2.0636960461856468e-12,
 1.0617861145221734e-07,
 0.1348418055921667,
 0.13484176042634835,
 1.5410626172102875e-12,
 0.1348381921622749,
 0.13484130611835682,
 0.1348417755390128,
 0.13483397331789718,
 0.1348417787843348,
 0.13483885754668484,
 3.0208984932527384e-12,
 2.2841452417601484e-12,
 1.7504903628648787e-12,
 0.13484145258426428,
 0.13484178367525435,
 0.13484180559477832,
 0.13483438498162822,
 2.336073272928913e-12,
 2.5395882397620416e-12,
 0.13484177809313813,
 0.1348351615406408,
 0.13484109003998637,
 0.13483244935222688,
 2.039146628427716e-12,
 0.13484178943281605,
 0.13483859850153793,
 0.1348410163959127,
 2.236735163552774e-12,
 0.13484180550104902,
 1.4446270476261423e-12,
 0.1348368391879147,
 1.6974377333154793e-12,
 0.13484167929029306,
 1.660201426573

In [30]:
a

,nodeId,embedding
0,15,"[0.13483924895325158, 2.71071429790426e-12, 1...."
1,16,"[0.13483924895325158, 2.71071429790426e-12, 1...."
2,17,"[0.13483924895325158, 2.71071429790426e-12, 1...."
3,18,"[0.13483924895325158, 2.71071429790426e-12, 1...."
4,19,"[0.13483924895325158, 2.71071429790426e-12, 1...."
...,...,...
413,1249,"[0.13483924895325158, 2.71071429790426e-12, 1...."
414,1250,"[0.13483924895325158, 2.71071429790426e-12, 1...."
415,1251,"[0.13483924895325158, 2.71071429790426e-12, 1...."
416,1252,"[0.13483924895325158, 2.71071429790426e-12, 1...."
